In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

sys.path.append(os.path.abspath('../src'))
import utils as util

# Definindo tema e cores
px.defaults.template = "plotly_white"
px.defaults.color_discrete_sequence = [util.barra_azul, util.barra_vermelha] 

df = pd.read_csv('../data/databaseElisa.csv', sep=';', decimal=',', thousands='.', 
                 index_col=None)

df.tail()


,data,fazenda,almoco,janta,cafe,lanche,vlrAlmoco,vlrJanta,vlrCafe,vlrLanche,total
3065,11/05/2025,Silo,5,6,5,0,20.0,20.0,8.5,8.5,262.5
3066,12/05/2025,Santa Elisa,5,1,6,1,20.0,20.0,8.5,8.5,179.5
3067,12/05/2025,Silo,12,6,14,0,20.0,20.0,8.5,8.5,479.0
3068,13/05/2025,Santa Elisa,9,2,6,1,20.0,20.0,8.5,8.5,279.5
3069,13/05/2025,Silo,12,6,14,0,20.0,20.0,8.5,8.5,479.0


In [2]:

df['data'] = pd.to_datetime(df['data'], dayfirst=True, errors='coerce')

data_inicio = pd.Timestamp('2025-02-01')


df_filtrado = df[df['data'] >= data_inicio].copy()


In [3]:
df_filtrado.head()

,data,fazenda,almoco,janta,cafe,lanche,vlrAlmoco,vlrJanta,vlrCafe,vlrLanche,total
2984,2025-04-01,Santa Elisa,6,1,0,0,20.0,20.0,8.5,8.5,140.0
2985,2025-04-01,Silo,25,7,25,2,20.0,20.0,8.5,8.5,869.5
2986,2025-04-02,Santa Elisa,7,1,0,0,20.0,20.0,8.5,8.5,160.0
2987,2025-04-02,Silo,21,8,25,4,20.0,20.0,8.5,8.5,826.5
2988,2025-04-03,Santa Elisa,6,1,0,0,20.0,20.0,8.5,8.5,140.0


In [4]:
df_filtrado.tail()

,data,fazenda,almoco,janta,cafe,lanche,vlrAlmoco,vlrJanta,vlrCafe,vlrLanche,total
3065,2025-05-11,Silo,5,6,5,0,20.0,20.0,8.5,8.5,262.5
3066,2025-05-12,Santa Elisa,5,1,6,1,20.0,20.0,8.5,8.5,179.5
3067,2025-05-12,Silo,12,6,14,0,20.0,20.0,8.5,8.5,479.0
3068,2025-05-13,Santa Elisa,9,2,6,1,20.0,20.0,8.5,8.5,279.5
3069,2025-05-13,Silo,12,6,14,0,20.0,20.0,8.5,8.5,479.0


In [5]:
util.generate_metadata(df_filtrado)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,data,datetime64[ns],0,0.0,43
1,almoco,int64,0,0.0,25
2,janta,int64,0,0.0,9
3,cafe,int64,0,0.0,15
4,lanche,int64,0,0.0,6
5,vlrAlmoco,float64,0,0.0,1
6,vlrJanta,float64,0,0.0,1
7,vlrCafe,float64,0,0.0,1
8,vlrLanche,float64,0,0.0,1
9,total,float64,0,0.0,64


In [6]:
# Descrição estatística das colunas numéricasdf
cols_to_describe = ['almoco', 'janta', 'cafe', 'lanche', 'vlrAlmoco', 'vlrJanta', 'vlrCafe', 'vlrLanche', 'total']

df_filtrado[cols_to_describe].describe()

,almoco,janta,cafe,lanche,vlrAlmoco,vlrJanta,vlrCafe,vlrLanche,total
count,86.000000,86.000000,86.000000,86.000000,86.0,86.0,86.0,86.0,86.000000
mean,10.023256,3.720930,10.732558,0.383721,20.0,20.0,8.5,8.5,369.372093
std,6.444289,2.742481,9.927294,1.294046,0.0,0.0,0.0,0.0,253.641644
min,0.000000,0.000000,0.000000,0.000000,20.0,20.0,8.5,8.5,0.000000
25%,5.000000,1.000000,0.000000,0.000000,20.0,20.0,8.5,8.5,160.000000
50%,9.000000,4.000000,8.500000,0.000000,20.0,20.0,8.5,8.5,342.000000
75%,14.750000,6.000000,20.000000,0.000000,20.0,20.0,8.5,8.5,590.000000
max,26.000000,8.000000,28.000000,10.000000,20.0,20.0,8.5,8.5,901.000000


In [7]:
util.lista_valores_unicos(df_filtrado, 'fazenda')

,Coluna,Categoria,Cardinalidade,Porcentagem
0,fazenda,Santa Elisa,43,50.00%
1,fazenda,Silo,43,50.00%


In [8]:
df_filtrado.columns = df_filtrado.columns.str.strip().str.lower()

In [9]:
# 002. Quantidade total de refeições por fazenda
qtd_cols = ['almoco', 'janta', 'lanche', 'cafe']
df_qtd_fazenda = df_filtrado.groupby('fazenda')[qtd_cols].sum().reset_index()
df_qtd_fazenda['total_refeicoes'] = df_qtd_fazenda[qtd_cols].sum(axis=1)

fig1 = px.bar(df_qtd_fazenda, x='fazenda', y='total_refeicoes', color='fazenda',
              title='Total de Refeições por Fazenda', text_auto=True)
fig1.show()

In [10]:
# 003. Evolução temporal do total de vendas
df_data_total = df_filtrado.groupby('data')['total'].sum().reset_index()
fig2 = px.line(df_data_total, x='data', y='total',
               title='Total de Vendas ao Longo do Tempo')
fig2.show()

In [11]:
# 004. Comparativo de ticket médio (valor unitário) entre fazendas
df_filtrado['vlr_unit_almoco'] = df_filtrado['vlralmoco'] / df_filtrado['almoco'].replace(0, pd.NA)
df_filtrado['vlr_unit_janta'] = df_filtrado['vlrjanta'] / df_filtrado['janta'].replace(0, pd.NA)
df_filtrado['vlr_unit_lanche'] = df_filtrado['vlrlanche'] / df_filtrado['lanche'].replace(0, pd.NA)
df_filtrado['vlr_unit_cafe'] = df_filtrado['vlrcafe'] / df_filtrado['cafe'].replace(0, pd.NA)

df_melt = df_filtrado.melt(id_vars=['fazenda'], value_vars=[
    'vlr_unit_almoco', 'vlr_unit_janta', 'vlr_unit_lanche', 'vlr_unit_cafe'],
    var_name='refeicao', value_name='vlr_unitario')

fig3 = px.box(df_melt, x='refeicao', y='vlr_unitario', color='fazenda',
              title='Distribuição do Valor Unitário por Tipo de Refeição e Fazenda')
fig3.show()

In [12]:
# 005. Análise sazonal: média de vendas por mês
df_filtrado['data'] = pd.to_datetime(df_filtrado['data'], dayfirst=True, errors='coerce')
df_filtrado['mes'] = df_filtrado['data'].dt.to_period('M').astype(str)
df_mes = df_filtrado.groupby('mes')['total'].mean().reset_index()

fig4 = px.line(df_mes, x='mes', y='total', markers=True,
               title='Média de Vendas por Mês')
fig4.update_xaxes(type='category')
fig4.show()

In [13]:
# 006. Análise percentual de participação das fazendas no total vendido
df_total_faz = df_filtrado.groupby('fazenda')['total'].sum().reset_index()
df_total_faz['percentual'] = df_total_faz['total'] / df_total_faz['total'].sum() * 100

fig5 = px.pie(df_total_faz, names='fazenda', values='percentual', color='fazenda',
              title='Participação de Cada Fazenda no Total Vendido (%)')
fig5.show()

In [14]:
# 007. Correlação entre quantidade e valor total das refeições
fig6 = px.scatter(df_filtrado, x='almoco', y='total', color='fazenda',
                  title='Quantidade x Valor Total (Almoço)')
fig6.show()

fig7 = px.scatter(df_filtrado, x='janta', y='total', color='fazenda',
                  title='Quantidade x Valor Total (Janta)')
fig7.show()

fig8 = px.scatter(df_filtrado, x='cafe', y='total', color='fazenda',
                  title='Quantidade x Valor Total (Cafe)')
fig8.show()

fig9 = px.scatter(df_filtrado, x='lanche', y='total', color='fazenda',
                  title='Quantidade x Valor Total (Lanche)')
fig9.show()

In [15]:
# 008. Evolução de cada tipo de refeição ao longo do tempo
df_refeicoes_tempo = df_filtrado.groupby('data')[qtd_cols].sum().reset_index()
fig8 = px.line(df_refeicoes_tempo, x='data', y=qtd_cols,
               title='Evolução Diária das Refeições por Tipo')
fig8.show()

In [16]:
# 009. Receita média por tipo de refeição
df_receita_media = pd.DataFrame({
    'Tipo': ['Almoço', 'Janta', 'Lanche', 'Café'],
    'Receita Média': [
        df_filtrado['vlralmoco'].mean(),
        df_filtrado['vlrjanta'].mean(),
        df_filtrado['vlrlanche'].mean(),
        df_filtrado['vlrcafe'].mean()
    ]
})
fig9 = px.bar(df_receita_media, x='Tipo', y='Receita Média', color='Tipo',
              title='Receita Média por Tipo de Refeição', text_auto='.2s')
fig9.show()

In [17]:
# 010. Análise de outliers no valor total por fazenda
fig10 = px.box(df_filtrado, x='fazenda', y='total',
               title='Distribuição do Valor Total por Fazenda')
fig10.show()

In [18]:
# 011. Proporção de cada tipo de refeição no total servido
df_total_refeicoes = df_filtrado[qtd_cols].sum().reset_index()
df_total_refeicoes.columns = ['Tipo', 'Total Servido']
df_total_refeicoes['Tipo'] = df_total_refeicoes['Tipo'].str.replace('qtd', '', regex=False).str.capitalize()

fig11 = px.pie(df_total_refeicoes, names='Tipo', values='Total Servido', color='Tipo',
               title='Proporção de Cada Tipo de Refeição Servida')
fig11.show()

In [19]:
# 012. Média diária de refeições por fazenda
df_media_diaria = df_filtrado.groupby('fazenda')[qtd_cols].mean().reset_index()
df_media_diaria = df_media_diaria.melt(id_vars='fazenda', var_name='Tipo', value_name='Média Diária')
df_media_diaria['Tipo'] = df_media_diaria['Tipo'].str.replace('qtd', '', regex=False).str.capitalize()

fig12 = px.bar(df_media_diaria, x='fazenda', y='Média Diária', color='Tipo',
               barmode='group', title='Média Diária de Refeições por Fazenda', text_auto=True)
fig12.show()

In [20]:
#013. Evolução do ticket médio diário por fazenda
df_filtrado['ticket_medio'] = df_filtrado['total'] / (df_filtrado[qtd_cols].sum(axis=1).replace(0, pd.NA))
df_ticket_diario = df_filtrado.groupby(['data', 'fazenda'])['ticket_medio'].mean().reset_index()

fig13 = px.line(df_ticket_diario, x='data', y='ticket_medio', color='fazenda',
                title='Evolução do Ticket Médio Diário por Fazenda')
fig13.show()


In [21]:
#014. Top 5 datas com maior venda total
df_top_vendas = df_filtrado.groupby('data')['total'].sum().nlargest(5).reset_index()

fig14 = px.bar(df_top_vendas, x='data', y='total',
               title='Top 5 Datas com Maior Venda Total', text_auto=True)
fig14.show()


In [22]:
#015. Dias da semana com maior venda média
df_filtrado['dia_semana'] = df_filtrado['data'].dt.day_name()
df_venda_dia = df_filtrado.groupby('dia_semana')['total'].mean().reset_index()

df_venda_dia['dia_semana'] = df_venda_dia['dia_semana'].map(util.traducao_dias)


ordem_dias_pt = ['Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 'Sexta-feira', 'Sábado', 'Domingo']
df_venda_dia['dia_semana'] = pd.Categorical(df_venda_dia['dia_semana'], categories=ordem_dias_pt, ordered=True)
df_venda_dia = df_venda_dia.sort_values('dia_semana')

fig15 = px.bar(df_venda_dia, x='dia_semana', y='total',
    title='Média de Vendas por Dia da Semana', text_auto=True)

fig15.show()

In [23]:

# 016. Média diária de quantidade refeições por dia da semana
qtd_cols = ['almoco', 'janta', 'lanche', 'cafe']
df_filtrado['dia_semana'] = df_filtrado['data'].dt.day_name()
df_media_semana = df_filtrado.groupby('dia_semana')[qtd_cols].mean().reset_index()

df_media_semana['dia_semana'] = df_media_semana['dia_semana'].map(util.traducao_dias)

ordem_dias_pt = ['Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 'Sexta-feira', 'Sábado', 'Domingo']
df_media_semana['dia_semana'] = pd.Categorical(df_media_semana['dia_semana'], categories=ordem_dias_pt, ordered=True)
df_media_semana = df_media_semana.sort_values('dia_semana')


df_melted = df_media_semana.melt(id_vars='dia_semana', var_name='Tipo', value_name='Média Diária')


df_melted['Tipo'] = df_melted['Tipo'].str.capitalize()


fig16 = px.bar(df_melted, x='dia_semana', y='Média Diária', color='Tipo', barmode='group',
    title='Média Diária de Refeições por Dia da Semana', text_auto=True)

fig16.show()